# Prelude

In [ ]:
# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:")
!git status --short

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..

# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from common import analysis, units, morphology, treeutils
import models.GilliesWillshaw.gillies_pynn_model as gillies_model
import neuron; h = neuron.h
import bluepyopt.ephys as ephys

# Load custom synapse mechanisms
neuron.load_mechanisms('/home/luye/workspace/bgcellmodels/mechanisms/synapses')

# Create cell

In [ ]:
# ! EphysModelWrapper instantiates the cell in __init__()
cell = gillies_model.StnCellModel()
icell = cell.icell
nrnsim = cell.sim

In [ ]:
named_seclists =  {listname: list(getattr(icell, listname)) for listname in cell.seclist_names}
for k, v in named_seclists.items():
    if len(v)==0:
        named_seclists.pop(k) # don't include empty SectionLists
    else:
        print("{} : {} sections".format(k, len(v)))

somatic = named_seclists['somatic']
dendritic = named_seclists['basal']

soma = somatic[0]
dend = dendritic[0]

## Synapse Locations

A suitable compartment can be selected either using `treeutils.subtree_topology` or by using 
NEURON GUI -> Tools -> Distributed Mechanisms > Viewers > Shape Name

In [ ]:
# inspect topology to determine good synapse location
# print(treeutils.subtree_topology(soma, max_depth=1))

# Pick segments on separate subtrees if you don't want simultaneous spikes to interfer
distal_seg = dendritic[8](0.75)
middle_seg = dendritic[20](0.25)
proximal_seg = dendritic[24](0.5)

In [ ]:
plt.figure()
ax = plt.subplot(111, projection='3d')
ax.view_init(90,90) # set azimut and elevation
ax.set_zlim((-1,1)) # prevent rounding errors
ax.set_axis_off() # hide x,y,z axes and grid

# Plot shape of our cell
h.define_shape(sec=soma) # for cells without 3D morphology
morphology.shapeplot(h, ax)

# Plot soma locations
markers = ('og', 'ob', 'or')
labels = ['prox', 'mid', 'dist']
for i, seg in enumerate((proximal_seg, middle_seg, distal_seg)):
    print(labels[i] + " : " + seg.sec.name())
    morphology.mark_locations(h, seg.sec, seg.x, label=labels[i], markspec=markers[i])

plt.legend()
plt.show()

# Excitatory Synapses

## Experimental Synapse Properties

For excitatory synapses (CTX -> STN), the paper by Chu (2015) about GP physiology shows AMPA and NMDA synapse properties in Figure 1.

**For AMPA**: Fig. 1E bottom left trace shows time course of an EPSC evoked when the cell is near its resting membrane potential:
- EPSC amplitude is ~ 275 pA
    + 275e-3 nA / (-80 mV - 0 mV) = -275e-3/80 uS = 3.44 nS
- time constant for rising phase (exponential) : ~ 1.0 ms
- time constant for decay phase (exponential) : ~ 4.0 ms


**For NMDA**: Fig. 1E top right trace shows time course of an EPSP evoked when the cell is at 40 mV which is close to the 30 mV at which Mg2+ block is removed from NMDAR.
- EPSC amplitude is ~ 210 pA
    + 210e-3 nA / (30 mV - 0 mV) = 210e-3 / 30 uS = 210/30 nS = 7 nS
- time constant for rising phase (exponential) : ~ 3.7 ms
- time constant for decay phase (exponential) : ~ 80.0 ms

Short term depression: time constant of 200 ms and base release probability of 0.7 fits data using Tsodyks-Markram model.
- TODO: find out which data, notes say Gradinaru 2009 but not found

## Results Summary (see below)

AMPA:
- Single synapse (no multi-synapse rule) with reported conductance of 3.44 nS yields plausible EPSP.

## Calibrate AMPA

In [ ]:
# Calibrate AMPA synapse to get desired PSP time course

spike_interval = 150.0
synapses, stims, netcons = [], [], []

conductance_nS = 3.44
tau_rise = 1.0
tau_decay = 4.0
tau_rec = 200.0
tau_facil = 1.0
P_release = 0.7

for i, seg in enumerate((proximal_seg, middle_seg, distal_seg)):
    syn = h.GLUsyn(seg)
    
    stim = h.NetStim()
    stim.number = 1
    stim.noise = 0
    stim.start = 750 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    #nc.weight[0] = conductance_nS / P_release # conductance in [nS]
    nc.weight[0] = 1.0
    syn.gmax_AMPA = conductance_nS / P_release * 1e-3 # [uS] or set netcon weight in [nS]
    syn.gmax_NMDA = 0.0
    syn.tau_r_AMPA = tau_rise
    syn.tau_d_AMPA = tau_decay
    
    syn.tau_rec = tau_rec
    syn.tau_facil = 1.0
    syn.U1 = P_release # release probability
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

# Disable spiking for more accurate PSP measurement
# for sec in h.allsec():
#     sec.gmax_NaF = 0.0

# Alternative to disable spiking: hyperpolarizing current (like in article figure)
istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = -0.2 # 100 pA = 0.1 nA

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'V_prox': {'var':'v', 'seg':'prox'},
    'V_mid': {'var':'v', 'seg':'mid'},
    'V_dist': {'var':'v', 'seg':'dist'},
}

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

## Calibrate NMDA

In [ ]:
# Calibrate NMDA synapse to get desired PSP time course
# NOTE: it is important to deliver spikes when Mg2+ gate is removed
#       e.g. by timing a pSP to arrive during a backpropagating spike
#       or after AMPA spikes

spike_interval = 150.0
synapses, stims, netcons = [], [], []

conductance_nS = 7.0
tau_rise = 3.7
tau_decay = 80.0
tau_rec = 200.0
tau_facil = 1.0
P_release = 0.7

for i, seg in enumerate((proximal_seg, middle_seg, distal_seg)):
    syn = h.GLUsyn(seg)
    
    stim = h.NetStim()
    stim.number = 1
    stim.noise = 0
    stim.start = 750 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    nc.weight[0] = conductance_nS / P_release # conductance in [nS]
    # syn.gmax_AMPA = 0.01 # [uS] or set netcon weight in [nS]
    syn.gmax_AMPA = 0.0
    syn.tau_r_NMDA = tau_rise
    syn.tau_d_NMDA = tau_decay
    
    syn.tau_rec = tau_rec
    syn.tau_facil = 1.0
    syn.U1 = P_release # release probability
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

# Disable spiking for more accurate PSP measurement
# for sec in h.allsec():
#     sec.gmax_NaF = 0.0

# Alternative to disable spiking: hyperpolarizing current (like in article figure)
istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = -0.2 # 100 pA = 0.1 nA

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'V_prox': {'var':'v', 'seg':'prox'},
    'V_mid': {'var':'v', 'seg':'mid'},
    'V_dist': {'var':'v', 'seg':'dist'},
}

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

# Inhibitory Synapse (GPe)

## Experimental Synapse Properties

For **GABA-A synapses** (GPe -> STN):

The paper by <span style='color:blue;font-weight:bold'> Chu (2015) </span> about GP physiology shows GABA-A ISPSc examples in Figures 2,3,4,5,6.

- IPSC amplitude is ~ 350 pA
    + 350e-3 nA / (-85 mV - -68 mV) = -350e-3/17 uS = 20.6 nS
- time constant for rising phase (exponential) : ~ 2.6 ms
- time constant for decay phase (exponential) : ~ 5.0 ms

The paper by <span style='color:blue;font-weight:bold'> Fan (2012) </span> shows GABA-A ISPSs and distribution of conductances in Fig. 2, and distributions of decay times in Fig. 1 (mISPC decay times should be same as for spike-induced IPSC).

- reported conductance is ~7 nS
    - median ~= 7 nS
    - sigma ~= 3 nS
- time constant for rising phase (exponential) : ~ 1.0 ms
- time constant for decay phase (exponential) : ~ 6.0 ms

The paper by <span style='color:blue;font-weight:bold'> Atherton (2013) </span> shows GABA-A IPSCs and short-term depression (STD) triggered by successive high-frequency IPSCs.

- exponential time constant of recovery is reported as 17.3 +/- 18.9 seconds
    - seee Fig 2B and legend
- time scale of depression:
    - Fig 2C-D: the x-axis means that the conductance distribution (min, mean, max) is for IPSCs falling between [i-1, i] seconds. So in the first second of 33 Hz stimulation the average amplitude is ~ 30% of peak amplitude, and in the second second it is ~ 12%.
    - example Fig. 1A, Fig 7A, 8A, 9A (control conditions)
- Fig 12 & 14 show magnitude of IPSPs for approx. 60 presynaptic inhibitory neurons foring synchronously
    - this indicates that indeed unitary conductances should be around 7 nS

For **GABA-B synapses** (GPe -> STN):

For inhibitory synapses (STR), the paper by Levine, Hull, Buchendwald (1974) about GP physiology shows following values in Figure 1:

- average IPSP trace shows approx. 5 mV dip from baseline
- exponential time constant for rising (dipping) phase : 66.0 ms
- exponential time constant for recovery phase : 100.0 ms

## Simulation Results

By cross-referencing and comparing data from three articles mentioned above, the conclusion is that the unitary conductance (single synapse) should approx. 7 nS as reported in Fan (2012). Hence for GABA-A we get:

- gsyn = 7 nS
- tau_rec = 17300.0
- tau_facil = 1.0
- P_release = 0.2


To calibrate a single GABAB synapse positioned at a proximal/trunk section, receiving volley of 8 spikes at 100 Hz to yield the IPSP shown in Figure 1, the following parameters work:

- gsyn = 0.0375 nS
- tau_rise_NT = 5.0 ms
- tau_decay_NT = 10.0 ms

In [ ]:
istim.amp = 0
istim = None

In [ ]:
# Calibrate GABAA conductance so we get PSP of desired magnitude

spike_interval = 150.0
synapses, stims, netcons = [], [], []

conductance_nS = 7.0
tau_rise = 2.0
tau_decay = 6.0
tau_rec = 17300.0
tau_facil = 1.0
P_release = 0.2

for i, seg in enumerate((proximal_seg,)):
    # Insert synapse in selected compartment
    syn = h.GABAsyn(seg)
    
    # Configure incoming spikes
    stim = h.NetStim()
    stim.number = 1e9 # need barrage to trigger signaling cascade
    stim_rate = 33.0 # Hz
    stim.interval = stim_rate**-1*1e3
    stim.noise = 0
    stim.start = 500 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    nc.weight[0] = conductance_nS / P_release # conductance in [nS]
    # syn.gmax_AMPA = 0.01 # [uS] or set netcon weight in [nS]
    syn.gmax_GABAB = 0.0
    syn.tau_r_GABAA = tau_rise
    syn.tau_d_GABAA = tau_decay
    
    syn.tau_rec = tau_rec
    syn.tau_facil = tau_facil
    syn.U1 = P_release # release probability
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
    'synGABA': synapses[0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'V_prox': {'var':'v', 'seg':'prox'},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

In [ ]:
# Calibrate GABAB conductance so we get PSP of desired magnitude

spike_interval = 0.0
synapses, stims, netcons = [], [], []

# NOTE: rise and decay time represent that of first quantity in signaling
#       cascade, NOT those of the conductance or IPSP
conductance_nS = 7.0
tau_rise = 5.0 # 66.0
tau_decay = 10.0 # 100.0

for i, seg in enumerate((proximal_seg,)):
    syn = h.GABAsyn(seg)
    
    stim = h.NetStim()
    stim.number = 8 # need barrage to trigger signaling cascade
    stim_rate = 100.0 # Hz
    stim.interval = stim_rate**-1*1e3
    stim.noise = 0
    stim.start = 500 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    hill_factor = 0.21
    nc.weight[0] = conductance_nS / hill_factor # conductance in [nS]
    # syn.gmax_AMPA = 0.01 # [uS] or set netcon weight in [nS]
    syn.gmax_GABAA = 0.0
    syn.tau_r_GABAB = tau_rise
    syn.tau_d_GABAB = tau_decay
    syn.U1 = 1.0 # release probability
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
    'synGABA': synapses[0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'V_prox': {'var':'v', 'seg':'prox'},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAB'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.set_aCSF(4) # Hoc function defined in Gillies code
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),